In [1]:
print("janani")


janani


In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("heart_disease_v1.csv")

# Quick check
print(df.head())
print(df.shape)


ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [9]:
!pip install pandas pytz


Defaulting to user installation because normal site-packages is not writeable


In [11]:
!pip install pytz --upgrade


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn

print("✅ Pandas:", pd.__version__)
print("✅ NumPy:", np.__version__)
print("✅ Matplotlib:", matplotlib.__version__)
print("✅ scikit-learn:", sklearn.__version__)


✅ Pandas: 2.2.3
✅ NumPy: 2.1.3
✅ Matplotlib: 3.10.1
✅ scikit-learn: 1.7.1


In [3]:
import pandas as pd

# Load dataset
df = pd.read_csv("heart_disease_v1.csv")

# Quick check
print(df.shape)
print(df.head())


(10, 14)
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  


In [4]:
# ========================================================
# 1. Import Libraries
# ========================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import joblib
import os

# ========================================================
# 2. Load Dataset
# ========================================================
df = pd.read_csv("heart_disease_v1.csv")

print("Dataset Shape:", df.shape)
print("First 5 rows:")
print(df.head())

# Features (X) and Target (y)
X = df.drop("target", axis=1)
y = df["target"]

# Split into Train & Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ========================================================
# 3. Logistic Regression with Regularization
# ========================================================

models = {
    "L1": LogisticRegression(penalty="l1", solver="saga", max_iter=5000, random_state=42),
    "L2": LogisticRegression(penalty="l2", solver="saga", max_iter=5000, random_state=42),
    "ElasticNet": LogisticRegression(penalty="elasticnet", solver="saga", max_iter=5000, random_state=42)
}

# Hyperparameter grids
param_grid = {
    "L1": {"C": [0.01, 0.1, 1, 10]},
    "L2": {"C": [0.01, 0.1, 1, 10]},
    "ElasticNet": {"C": [0.01, 0.1, 1, 10], "l1_ratio": [0.1, 0.5, 0.9]}
}

# To store results
results = {}

# ========================================================
# 4. Training & Evaluation
# ========================================================
for name, model in models.items():
    print(f"\n🔹 Training {name} Logistic Regression...")
    grid = GridSearchCV(model, param_grid[name], cv=5, scoring="f1", n_jobs=-1)
    grid.fit(X_train, y_train)
    
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]
    
    results[name] = {
        "best_params": grid.best_params_,
        "classification_report": classification_report(y_test, y_pred, output_dict=True),
        "roc_auc": roc_auc_score(y_test, y_proba)
    }
    
    # Save trained model
    os.makedirs("models", exist_ok=True)
    joblib.dump(best_model, f"models/log_reg_{name.lower()}.pkl")

# ========================================================
# 5. Save Results
# ========================================================
results_df = pd.DataFrame({
    model: {
        "Best Params": results[model]["best_params"],
        "ROC-AUC": results[model]["roc_auc"],
        "F1-Score": results[model]["classification_report"]["1"]["f1-score"]
    }
    for model in results
}).T

results_df.to_csv("logistic_regression_results.csv", index=True)

print("\n✅ Final Results:")
print(results_df)


Dataset Shape: (10, 14)
First 5 rows:
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  

🔹 Training L1 Logistic Regression...


C:\Users\student\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



🔹 Training L2 Logistic Regression...

🔹 Training ElasticNet Logistic Regression...


C:\Users\student\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\student\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



✅ Final Results:
                            Best Params ROC-AUC F1-Score
L1                             {'C': 1}     0.0      0.0
L2                          {'C': 0.01}     0.0      0.0
ElasticNet  {'C': 0.1, 'l1_ratio': 0.1}     0.0      0.0


In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("student_scores_v1.csv")

# Quick check
print(df.shape)
print(df.head())

(10, 2)
   Hours_Studied  Score
0              1     10
1              2     25
2              3     35
3              4     50
4              5     65


In [3]:
# ========================================================
# 1. Import Libraries
# ========================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os

# ========================================================
# 2. Dataset (Already Loaded)
# ========================================================
# Assuming your dataset is loaded as df
# df = pd.read_csv("student_scores_v1.csv")

print("Dataset Shape:", df.shape)
print(df.head())

# Separate features & target
X = df.drop("Score", axis=1)   # feature(s), adjust column name if needed
y = df["Score"]                # target

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ========================================================
# 3. Polynomial Regression
# ========================================================

# Polynomial transformation (degree=2, can try 3)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Train Linear Regression on transformed features
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

# Predictions
y_pred = poly_model.predict(X_test_poly)

# ========================================================
# 4. Evaluation
# ========================================================
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n✅ Model Evaluation")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

# ========================================================
# 5. MLOps Practices: Save Model + Preprocessor
# ========================================================
os.makedirs("models", exist_ok=True)

joblib.dump(poly, "models/poly_transformer.pkl")
joblib.dump(poly_model, "models/polynomial_regression.pkl")

# Save metrics for tracking
results = {
    "model": "Polynomial Regression",
    "degree": 2,
    "mse": mse,
    "r2": r2
}
results_df = pd.DataFrame([results])
results_df.to_csv("polynomial_regression_results.csv", index=False)

print("\n✅ Model and results saved!")


Dataset Shape: (10, 2)
   Hours_Studied  Score
0              1     10
1              2     25
2              3     35
3              4     50
4              5     65

✅ Model Evaluation
Mean Squared Error: 0.6272
R2 Score: 0.9995

✅ Model and results saved!
